In [20]:
import sys
sys.path.append("../pyvene/")

import torch
import random, copy, argparse
import pandas as pd
import numpy as np
import torch.nn.functional as F
import seaborn as sns
from tqdm import tqdm, trange
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer
from torch.nn import CrossEntropyLoss
from transformers.activations import ACT2FN
import wandb

from pyvene import (
    IntervenableModel,
    LowRankRotatedSpaceIntervention,
    RepresentationConfig,
    IntervenableConfig,
    ConstantSourceIntervention,
    TrainableIntervention,
    DistributedRepresentationIntervention,
)
from pyvene import create_llama
from pyvene import set_seed, count_parameters
from pyvene.models.layers import LowRankRotateLayer

import io
import json
import os
import re

def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode)
    return f

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

def jdump(obj, f, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.

    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()

def create_directory(path):
    """Create directory if not exist"""
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory '{path}' created successfully.")
    else:
        print(f"Directory '{path}' already exists.")

def extract_answer_number(sentence: str) -> float:
    sentence = sentence.replace(',', '')
    pred = [s for s in re.findall(r'-?\d+\.?\d*', sentence)]
    if not pred:
        return float('inf')
    pred_answer = float(pred[-1])
    if isinstance(pred_answer, str):
        try:
            pred_answer = float(pred_answer)
        except ValueError as e:
            pred_answer = float('inf')
    return pred_answer


def extract_answer_letter(sentence: str) -> str:
    sentence_ = sentence.strip()
    pred_answers = re.findall(r'A|B|C|D|E', sentence_)
    if pred_answers:
        if not pred_answers:
            return ''
        return pred_answers[-1]
    else:
        return ''
        
device = "cuda"
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"
prompt_template = """Below is an instruction that \
describes a task. Write a response that appropriately \
completes the request.

### Instruction:
%s

### Response:
"""
trigger_tokens = "### Response:\n"

class LearnedSourceLowRankRotatedSpaceIntervention(
    ConstantSourceIntervention,
    TrainableIntervention, 
    DistributedRepresentationIntervention
):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        rotate_layer = LowRankRotateLayer(self.embed_dim, kwargs["low_rank_dimension"])
        self.rotate_layer = torch.nn.utils.parametrizations.orthogonal(rotate_layer)
        self.learned_source = torch.nn.Parameter(
            torch.rand(kwargs["low_rank_dimension"]), requires_grad=True)
        
    def forward(
        self, base, source=None, subspaces=None
    ):
        rotated_base = self.rotate_layer(base)
        output = base + torch.matmul(
            (self.learned_source - rotated_base), self.rotate_layer.weight.T
        )
        return output.to(base.dtype)

class ConditionedSourceLowRankRotatedSpaceIntervention(
    ConstantSourceIntervention,
    TrainableIntervention, 
    DistributedRepresentationIntervention
):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        rotate_layer = LowRankRotateLayer(self.embed_dim, kwargs["low_rank_dimension"])
        self.rotate_layer = torch.nn.utils.parametrizations.orthogonal(rotate_layer)
        self.learned_source = torch.nn.Linear(
            self.embed_dim, kwargs["low_rank_dimension"]).to(torch.bfloat16)
        self.act_fn = ACT2FN["silu"]
        
    def forward(
        self, base, source=None, subspaces=None
    ):
        rotated_base = self.rotate_layer(base)
        output = base + torch.matmul(
            (self.act_fn(self.learned_source(base)) - rotated_base), self.rotate_layer.weight.T
        )
        return output.to(base.dtype)
    
class ConditionedSourceLowRankIntervention(
    ConstantSourceIntervention,
    TrainableIntervention, 
    DistributedRepresentationIntervention
):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.proj_layer = torch.nn.Linear(
            self.embed_dim, kwargs["low_rank_dimension"]).to(torch.bfloat16)
        self.learned_source = torch.nn.Linear(
            self.embed_dim, kwargs["low_rank_dimension"]).to(torch.bfloat16)
        self.act_fn = ACT2FN["silu"]
        
    def forward(
        self, base, source=None, subspaces=None
    ):
        proj_base = self.proj_layer(base)
        output = base + torch.matmul(
            (self.act_fn(self.learned_source(base)) - proj_base), self.proj_layer.weight
        )
        return output.to(base.dtype)
    

In [2]:
config, _, llama = create_llama("huggyllama/llama-7b")
_ = llama.to(device)  # single gpu
_ = llama.eval()  # always no grad on the model

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

loaded model


In [3]:
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("huggyllama/llama-7b")
tokenizer.padding_side = "right" 
tokenizer.pad_token = tokenizer.eos_token

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [15]:
task = "math_10k"

###################
# data loaders
###################
all_base_input_ids, all_base_positions, all_output_ids, all_source_input_ids = [], [], [], []

task_dataset = jload(f"./datasets/{task}/train.json")
for data_item in task_dataset:
    assert data_item['input'] == ""
    base_prompt = prompt_template % data_item['instruction'].strip()
    # base input = base prompt + steered base output
    base_input = base_prompt + data_item["output"].strip() + tokenizer.pad_token
    base_prompt_length = len(tokenizer(
        # we use 256 to follow previous work's cut-off length
        base_prompt, max_length=256, truncation=True, return_tensors="pt")["input_ids"][0])
    base_input_ids = tokenizer(
        base_input, max_length=256, truncation=True, return_tensors="pt")["input_ids"][0]
    output_ids = tokenizer(
        base_input, max_length=256, truncation=True, return_tensors="pt")["input_ids"][0]
    base_input_ids[-1] = tokenizer.pad_token_id
    output_ids[-1] = tokenizer.pad_token_id
    output_ids[:base_prompt_length] = -100
    
    all_base_input_ids.append(base_input_ids)
    all_base_positions.append([base_prompt_length-1]) # intervene on the last prompt token
    all_output_ids.append(output_ids)
    
raw_train = (
    all_base_input_ids,
    all_base_positions,
    all_output_ids,
)

train_dataset = Dataset.from_dict(
    {
        "input_ids": raw_train[0],
        "intervention_position": raw_train[1],
        "labels": raw_train[2],
    }
).shuffle(seed=42)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=llama,
    label_pad_token_id=-100,
    padding="longest"
)

train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)

In [16]:
epochs = 1
initial_lr = 8e-3
total_step = 0
gradient_accumulation_steps = 2
batch_size = 8

train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=batch_size, collate_fn=data_collator)

In [21]:
rank = 4
layers = "4;10;12;18"
layers = [int(l) for l in layers.split(";")]*2
    
config = IntervenableConfig([{
    "layer": l,
    "component": "block_output",
    "low_rank_dimension": rank} for l in layers],
    ConditionedSourceLowRankIntervention
)
intervenable = IntervenableModel(config, llama)
intervenable.set_device(device)
intervenable.disable_model_gradients()

In [ ]:
optimizer = torch.optim.Adam(
    intervenable.get_trainable_parameters(), lr=initial_lr
)
intervenable.model.train()  # train enables drop-off but no grads
print("llama trainable parameters: ", count_parameters(intervenable.model))
print("intervention trainable parameters: ", intervenable.count_parameters())
train_iterator = trange(0, int(epochs), desc="Epoch")
for epoch in train_iterator:
    epoch_iterator = tqdm(
        train_dataloader, desc=f"Epoch: {epoch}", position=0, leave=True
    )
    for step, inputs in enumerate(epoch_iterator):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(device)
        b_s = inputs["input_ids"].shape[0]
        
        base_unit_location = inputs["intervention_position"].tolist()
        base_first_token = torch.zeros_like(inputs["intervention_position"]).tolist()
        _, cf_outputs = intervenable(
            {"input_ids": inputs["input_ids"]},
            unit_locations={"sources->base": (None, [
                base_first_token
            ]*(len(layers)//2)+[
                base_unit_location
            ]*(len(layers)//2))})

        # lm loss on counterfactual labels
        lm_logits = cf_outputs.logits
        labels = inputs["labels"]
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        # Flatten the tokens
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        loss_str = round(loss.item(), 2)
        epoch_iterator.set_postfix({"loss": loss_str})
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        loss.backward()
        if total_step % gradient_accumulation_steps == 0:
            if not (gradient_accumulation_steps > 1 and total_step == 0):
                optimizer.step()
                optimizer.zero_grad()
        total_step += 1

llama trainable parameters:  0
intervention trainable parameters:  262208


Epoch: 0:  27%|███████████████████████████████████████████████▍                                                                                                                              | 338/1240 [03:23<09:16,  1.62it/s, loss=0.49]

In [46]:
problem = (
    "Paige had 27 files on her computer. She deleted 9 of them and put the rest into folders with 6 files in each one. How many folders did Paige end up with?"
)

prompt = prompt_template % (problem)
# base input = base prompt + steered base output
answer = 15.0

prompt = tokenizer(prompt, return_tensors="pt").to(device)
base_unit_location = prompt["input_ids"].shape[-1] - 1 
base_unit_location = {"sources->base": (
    None, [[[0]]]*(len(layers)//2) + [[[base_unit_location]]]*(len(layers)//2))}

_, steered_response = intervenable.generate(
    prompt, 
    unit_locations=base_unit_location,
    intervene_on_prompt=True,
    max_new_tokens=512, do_sample=False, 
    eos_token_id=tokenizer.pad_token_id, early_stopping=True
)
raw_generation = tokenizer.decode(steered_response[0], skip_special_tokens=True)
generation = raw_generation.split(trigger_tokens)[1]

generation = generation.strip()
if eval_tasks[i] == "AQuA":
    generation = extract_answer_letter(generation)
else:
    generation = extract_answer_number(generation)

print(raw_generation, generation, answer)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Paige had 27 files on her computer. She deleted 9 of them and put the rest into folders with 6 files in each one. How many folders did Paige end up with?

### Response:
To solve this problem, we need to use the following steps:

1. Find the total number of files Paige had before deleting any.
2. Find the total number of files Paige had after deleting 9 files.
3. Find the total number of files Paige had after putting the remaining files into folders.
4. Find the total number of folders Paige ended up with.

Step 1: Find the total number of files Paige had before deleting any.

Paige had 27 files before deleting any.

Step 2: Find the total number of files Paige had after deleting 9 files.

Paige had 27 files before deleting any.

Paige deleted 9 files, so she had 27 - 9 = 18 files left.

Step 3: Find the 3.0 15.0


In [30]:
prompt = prompt_template % (problem)

In [39]:
eval_tasks = [
    "MultiArith", 
    # "gsm8k", "AddSub", "AQuA", "SingleEq", "SVAMP"
]
for i in range(len(eval_tasks)):
    eval_dataset = jload(f"./datasets/{eval_tasks[i]}/test.json")
    sampled_eval_dataset = eval_dataset
    correct_count = 0
    totol_count = 0
    eval_iterator = tqdm(
        sampled_eval_dataset, position=0, leave=True
    )
    for data_item in eval_iterator:
        prompt = prompt_template % (data_item['instruction'])
        # base input = base prompt + steered base output
        answer = data_item["answer"]
        prompt = tokenizer(prompt, return_tensors="pt").to(device)
        base_unit_location = prompt["input_ids"].shape[-1] - 1 
        base_unit_location = {"sources->base": (
            None, [[[0]]]*(len(layers)//2) + [[[base_unit_location]]]*(len(layers)//2))}
        _, steered_response = intervenable.generate(
            prompt, 
            unit_locations=base_unit_location,
            intervene_on_prompt=True,
            max_new_tokens=512, do_sample=False, 
            eos_token_id=tokenizer.pad_token_id, early_stopping=True
        )
        raw_generation = tokenizer.decode(steered_response[0], skip_special_tokens=True)
        generation = raw_generation.split(trigger_tokens)[1]
        
        generation = generation.strip()
        if eval_tasks[i] == "AQuA":
            generation = extract_answer_letter(generation)
            if generation.strip() == answer.strip():
                correct_count += 1
        else:
            generation = extract_answer_number(generation)
            if generation == float(answer):
                correct_count += 1
        totol_count += 1
        print(correct_count/totol_count)

  0%|▎                                                                                                                                                                                                   | 1/600 [00:12<2:00:09, 12.04s/it]

1.0


  0%|▋                                                                                                                                                                                                   | 2/600 [00:25<2:06:15, 12.67s/it]

0.5


  0%|▉                                                                                                                                                                                                   | 3/600 [00:36<1:59:11, 11.98s/it]

...

0.45454545454545453


  2%|███▉                                                                                                                                                                                               | 12/600 [02:26<2:00:14, 12.27s/it]

0.4166666666666667


  2%|███▉                                                                                                                                                                                               | 12/600 [02:36<2:07:48, 13.04s/it]


KeyboardInterrupt: 

In [44]:
eval_dataset[1]

{'instruction': ' Bianca was planting vegetables in her garden. She started with 52 seeds and planted 40 of them in the big garden and in each of her small gardens put 2 seeds each. How many small gardens did Bianca have? ',
 'input': '',
 'output': '\nA: Bianca planted 40 seeds in the big garden. She put 2 seeds in each of her small gardens. That means she had 40 / 2 = 20 small gardens. The answer is 20.',
 'answer': '6.0'}